<details>
<summary>Table of Contents</summary>

- [1. Standard Scaler Informer ](#1-standard-scaler-informer)
- [2. Standard Scaler PatchTST](#2-standard-scaler-patchtst)
- [3. MinMax (0, 1) Scaler Informer](#3-minmax-scaler-0-1-informer)
- [4. MinMax (0, 1) Scaler PatchTST](#4-minmax-scaler-0-1-patchtst)
- [5. MinMax (0, 5) Scaler Informer](#5-minmax-scaler-0-5-informer)
- [6. MinMax (0, 5) Scaler PatchTST](#6-minmax-scaler-0-5-patchtst)

</details>


Here we perform a check on DE dataset to confirm choice of loss function and scaler for our data.

This script is to run the models. Final results are in the notebook "Comparison". 

Please note, the cell content is almost identical. However, when duplicating code and changing some arguments, it becomes easier to store and read results (especially if you want to experiment with 1 subpart) and split long running time into subprocesses. 

In [3]:
import os
import pandas as pd
import subprocess
import shutil
from utils.helper import extract_metrics_from_output, convert_results_into_df

# 1. Robust Scaler Informer

In [4]:
# Paths to files and data
data_path = os.getcwd() + "/datasets/"

script_path = "./PatchTST-main/PatchTST_supervised/run_longExp.py"

# Arguments that will be used also for file names
model = "Informer"
dataset = 'DE_data.csv'
losses = ["MSE", "RMSE", "MAE"]
country = dataset[:2]

log_dir = f"logs/loss_choice/robust"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

log_file_path = f"{log_dir}/{model}_{country}.log"

In [5]:
# Dynamic variables
pred_lens = ["24", "96", "168"]
seq_len = "96"
lr = "0.0001"
itr = 2  
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Lists to store the results
informer_results_scaled, informer_results_unscaled = [], []

# Log file
with open(log_file_path, "w") as log_file:
    for loss in losses:
        statement_1 = f"\n=== Starting experiments for loss function: {loss} ===\n"
        log_file.write(statement_1)
        print(statement_1)  # Print to .ipynb output cell

        for pred_len in pred_lens:
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2) 
            model_id = f"{country}_{seq_len}_{pred_len}_loss_choice_for_{country}"

            # Arguments for the command
            command = f"""
            python {script_path} \
              --random_seed 2021 \
              --is_training 1 \
              --root_path "{data_path}" \
              --data_path "{dataset}" \
              --model_id {model_id} \
              --model "{model}" \
              --data "custom" \
              --features M \
              --seq_len {seq_len} \
              --label_len 5 \
              --pred_len {pred_len} \
              --e_layers 2 \
              --d_layers 1 \
              --factor 5 \
              --enc_in 5 \
              --dec_in 5 \
              --c_out 5 \
              --des 'Exp' \
              --train_epochs 10 \
              --patience 3 \
              --overlapping_windows \
              --inverse \
              --loss_fnc "{loss}" \
              --scaler_type robust \
              --itr {itr} --batch_size 32 --learning_rate "{lr}"
            """

            # Run the command and capture the output
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture the output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')  # Print in the .ipynb cell
                log_file.write(line)  # Write to the log file

            # Wait for the process to complete
            process.wait()
            
            # Delete the checkpoints folder and all its contents
            shutil.rmtree('./checkpoints' )

            # Extract metrics for each iteration
            iteration_metrics = extract_metrics_from_output(output, itr)
            iteration_metrics_unscaled = extract_metrics_from_output(output, itr, if_scaled=False)

            # Log the extracted metrics and save them
            for iteration, (scaled_metrics, unscaled_metrics) in enumerate(zip(iteration_metrics, iteration_metrics_unscaled), start=1):
                log_file.write(f"\nExtracted Metrics for {country}, pred_len={pred_len}, iteration={iteration}:\n")
                log_file.write(f"Scaled Metrics - MSE: {scaled_metrics[0]}, RMSE: {scaled_metrics[1]}, MAE: {scaled_metrics[2]}, RSE: {scaled_metrics[3]}\n")
                log_file.write(f"Unscaled Metrics - MSE: {unscaled_metrics[0]}, RMSE: {unscaled_metrics[1]}, MAE: {unscaled_metrics[2]}, RSE: {unscaled_metrics[3]}\n")

                # Append the results to the informer_results lists
                metrics_data = [(informer_results_scaled, scaled_metrics), (informer_results_unscaled, unscaled_metrics)]

                for result_list, metrics in metrics_data:
                    result_list.append({
                        'Loss_function': loss,
                        'Pred_len': pred_len,
                        'Iteration': iteration,
                        'MSE': metrics[0],
                        'RMSE': metrics[1],
                        'MAE': metrics[2],
                        'RSE': metrics[3]
                    })


=== Starting experiments for loss function: MSE ===


=== Starting experiments for pred_len: 24 ===

Args in experiment:
Namespace(random_seed=2021, is_training=1, model_id='DE_96_24_loss_choice_for_DE', model='Informer', data='custom', root_path='/vol/fob-vol3/nebenf24/riabchuv/my_work/datasets/', data_path='DE_data.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', overlapping_windows=True, scaler_type='robust', seq_len=96, label_len=5, pred_len=24, inverse=True, loss_fnc='MSE', if_relu=False, fc_dropout=0.05, head_dropout=0.0, patch_len=16, stride=8, padding_patch='end', revin=1, affine=0, subtract_last=0, decomposition=0, kernel_size=25, individual=0, embed_type=0, enc_in=5, dec_in=5, c_out=5, d_model=512, n_heads=8, e_layers=2, d_layers=1, d_ff=2048, moving_avg=25, factor=5, distil=True, dropout=0.05, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers=10, itr=2, train_epochs=10, batch_size=32, patience=3, learning_rate=0.

In [6]:
path_dir = './dataset_results'
csv_name_scaled = 'informer_loss_functions_results_scaled_robust.csv'
csv_name_unscaled = 'informer_loss_functions_results_unscaled_robust.csv'

# Convert the results into a DataFrame and save as CSV
informer_df_scaled = convert_results_into_df(informer_results_scaled, path_dir, csv_name_scaled)
informer_df_unscaled = convert_results_into_df(informer_results_unscaled, path_dir, csv_name_unscaled)
informer_df_scaled.round(4)

MSE    RMSE     MAE     RSE
Loss_function Iteration Pred_len                                
MSE           1         24        0.2589  0.5088  0.3583  0.4946
              2         24        0.2400  0.4899  0.3382  0.4762
              1         96        0.4419  0.6648  0.4733  0.6466
              2         96        0.5003  0.7073  0.5115  0.6880
              1         168       0.4673  0.6836  0.5047  0.6628
              2         168       0.4636  0.6809  0.5041  0.6602
RMSE          1         24        0.2627  0.5125  0.3597  0.4982
              2         24        0.2452  0.4952  0.3417  0.4813
              1         96        0.4388  0.6624  0.4693  0.6443
              2         96        0.4716  0.6867  0.4867  0.6679
              1         168       0.4610  0.6790  0.5002  0.6583
              2         168       0.4807  0.6933  0.5034  0.6723
MAE           1         24        0.2681  0.5178  0.3306  0.5033
              2         24        0.2553  0.5053  0.3241  0.4912
              1         96        0.4912  0.7008  0.4737  0.6816
              2         96        0.4662  0.6828  0.4586  0.6641
              1         168       0.5403  0.7351  0.5003  0.7128
              2         168       0.5429  0.7368  0.5033  0.7145

In [7]:
informer_df_unscaled.round(4)

MSE       RMSE        MAE     RSE
Loss_function Iteration Pred_len                                          
MSE           1         24        20458642.0  4523.1230  3016.6570  0.2249
              2         24        18265502.0  4273.8159  2791.9541  0.2125
              1         96        36498908.0  6041.4326  3972.6138  0.3009
              2         96        42418092.0  6512.9175  4375.4331  0.3243
              1         168       39235132.0  6263.7954  4307.7930  0.3121
              2         168       37890544.0  6155.5298  4260.3516  0.3067
RMSE          1         24        20469224.0  4524.2925  3007.1609  0.2250
              2         24        18739230.0  4328.8833  2822.3328  0.2152
              1         96        35833200.0  5986.0840  3917.4497  0.2981
              2         96        39601956.0  6293.0083  4131.7314  0.3134
              1         168       38069076.0  6170.0142  4236.1050  0.3074
              2         168       39885020.0  6315.4590  4252.6626  0.3147
MAE           1         24        20925334.0  4574.4219  2752.5798  0.2274
              2         24        19889488.0  4459.7632  2691.1169  0.2217
              1         96        40326344.0  6350.3027  3975.0549  0.3162
              2         96        37853992.0  6152.5596  3816.9463  0.3064
              1         168       44810908.0  6694.0952  4207.5996  0.3335
              2         168       44440940.0  6666.4038  4218.4233  0.3322

In [8]:
# Uncomment the following lines if you want to read saved results
#path_dir = './dataset_results'
#csv_name_scaled = 'informer_loss_functions_results_scaled.csv'
#csv_name_unscaled = 'informer_loss_functions_results_unscaled.csv'

# Average the iterations
informer_scaled = pd.read_csv(os.path.join(path_dir, csv_name_scaled))
informer_unscaled = pd.read_csv(os.path.join(path_dir, csv_name_unscaled))

inf_res_scaled = informer_scaled.groupby(['Pred_len', 'Loss_function']).mean().drop('Iteration', axis=1)
inf_res_unscaled = informer_unscaled.groupby(['Pred_len', 'Loss_function']).mean().sort_index().drop('Iteration', axis=1)
inf_res_scaled.round(4)

MSE    RMSE     MAE     RSE
Pred_len Loss_function                                
24       MAE            0.2617  0.5115  0.3273  0.4972
         MSE            0.2494  0.4994  0.3483  0.4854
         RMSE           0.2539  0.5039  0.3507  0.4898
96       MAE            0.4787  0.6918  0.4661  0.6729
         MSE            0.4711  0.6860  0.4924  0.6673
         RMSE           0.4552  0.6746  0.4780  0.6561
168      MAE            0.5416  0.7360  0.5018  0.7136
         MSE            0.4654  0.6822  0.5044  0.6615
         RMSE           0.4708  0.6861  0.5018  0.6653

In [9]:
inf_res_unscaled.round(4)

MSE       RMSE        MAE     RSE
Pred_len Loss_function                                          
24       MAE            20407411.0  4517.0925  2721.8484  0.2246
         MSE            19362072.0  4398.4695  2904.3055  0.2187
         RMSE           19604227.0  4426.5879  2914.7468  0.2201
96       MAE            39090168.0  6251.4312  3896.0006  0.3113
         MSE            39458500.0  6277.1750  4174.0234  0.3126
         RMSE           37717578.0  6139.5461  4024.5906  0.3058
168      MAE            44625924.0  6680.2495  4213.0115  0.3328
         MSE            38562838.0  6209.6626  4284.0723  0.3094
         RMSE           38977048.0  6242.7366  4244.3838  0.3110

# 2. Robust Scaler PatchTST

In [10]:
# Dynamic variables
pred_lens = ["24", "96", "168"]
seq_len = "512"
lr = "0.0001"
model = "PatchTST"
itr = 2  
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# New log file path
log_file_path = f"{log_dir}/{model}_{country}.log"

patchtst_results_scaled, patchtst_results_unscaled = [], []

with open(log_file_path, "w") as log_file:
    for loss in losses:
        statement_1 = f"\n=== Starting experiments for loss function: {loss} ===\n"
        log_file.write(statement_1)
        print(statement_1)  # Print to notebook

        for pred_len in pred_lens:
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2) 
            model_id = f"{country}_{seq_len}_{pred_len}_loss_choice_for_{country}"

            # Command arguments
            command = f"""
            python {script_path} \
              --random_seed 2021 \
              --is_training 1 \
              --root_path "{data_path}" \
              --data_path "{dataset}" \
              --model_id {model_id} \
              --model "{model}" \
              --data "custom" \
              --features M \
              --seq_len {seq_len} \
              --label_len 5 \
              --pred_len {pred_len} \
              --e_layers 2 \
              --d_layers 1 \
              --factor 5 \
              --enc_in 5 \
              --dec_in 5 \
              --c_out 5 \
              --des 'Exp' \
              --train_epochs 10 \
              --patience 3 \
              --overlapping_windows \
              --inverse \
              --scaler_type robust \
              --loss_fnc "{loss}" \
              --itr {itr} --batch_size 32 --learning_rate "{lr}"
            """

            # Run the command and capture the output
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture the output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')  # Print in the .ipynb cell
                log_file.write(line)  # Write to the log file

            # Wait for the process to complete
            process.wait()

            # Delete the checkpoints folder and all its contents
            shutil.rmtree('./checkpoints' )

            # Extract metrics for each iteration
            iteration_metrics_scaled = extract_metrics_from_output(output, itr)
            iteration_metrics_unscaled = extract_metrics_from_output(output, itr, if_scaled=False)

            # Log the extracted metrics and save them
            for iteration, (scaled_metrics, unscaled_metrics) in enumerate(zip(iteration_metrics_scaled, iteration_metrics_unscaled), start=1):
                log_file.write(f"\nExtracted Metrics for {country}, pred_len={pred_len}, iteration={iteration}:\n")
                log_file.write(f"Scaled Metrics - MSE: {scaled_metrics[0]}, RMSE: {scaled_metrics[1]}, MAE: {scaled_metrics[2]}, RSE: {scaled_metrics[3]}\n")
                log_file.write(f"Unscaled Metrics - MSE: {unscaled_metrics[0]}, RMSE: {unscaled_metrics[1]}, MAE: {unscaled_metrics[2]}, RSE: {unscaled_metrics[3]}\n")

                # Append the results to the informer_results lists
                metrics_data = [(patchtst_results_scaled, scaled_metrics), (patchtst_results_unscaled, unscaled_metrics)]

                for result_list, metrics in metrics_data:
                    result_list.append({
                        'Loss_function': loss,
                        'Pred_len': pred_len,
                        'Iteration': iteration,
                        'MSE': metrics[0],
                        'RMSE': metrics[1],
                        'MAE': metrics[2],
                        'RSE': metrics[3]
                    })


=== Starting experiments for loss function: MSE ===


=== Starting experiments for pred_len: 24 ===

Args in experiment:
Namespace(random_seed=2021, is_training=1, model_id='DE_512_24_loss_choice_for_DE', model='PatchTST', data='custom', root_path='/vol/fob-vol3/nebenf24/riabchuv/my_work/datasets/', data_path='DE_data.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', overlapping_windows=True, scaler_type='robust', seq_len=512, label_len=5, pred_len=24, inverse=True, loss_fnc='MSE', if_relu=False, fc_dropout=0.05, head_dropout=0.0, patch_len=16, stride=8, padding_patch='end', revin=1, affine=0, subtract_last=0, decomposition=0, kernel_size=25, individual=0, embed_type=0, enc_in=5, dec_in=5, c_out=5, d_model=512, n_heads=8, e_layers=2, d_layers=1, d_ff=2048, moving_avg=25, factor=5, distil=True, dropout=0.05, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers=10, itr=2, train_epochs=10, batch_size=32, patience=3, learning_rate=

In [11]:
path_dir = './dataset_results'
csv_name_scaled = 'patchtst_loss_functions_results_scaled_robust.csv'
csv_name_unscaled = 'patchtst_loss_functions_results_unscaled_robust.csv'


# Convert the results into a DataFrame and save as CSV
patchtst_df_scaled = convert_results_into_df(patchtst_results_scaled, path_dir, csv_name_scaled)
patchtst_df_unscaled = convert_results_into_df(patchtst_results_unscaled, path_dir, csv_name_unscaled)
patchtst_df_scaled.round(4)

MSE    RMSE     MAE     RSE
Loss_function Iteration Pred_len                                
MSE           1         24        0.2438  0.4937  0.3271  0.4799
              2         24        0.2341  0.4838  0.3247  0.4703
              1         96        0.4056  0.6369  0.4533  0.6195
              2         96        0.4231  0.6505  0.4594  0.6326
              1         168       0.4300  0.6557  0.4732  0.6358
              2         168       0.4409  0.6640  0.4774  0.6439
RMSE          1         24        0.2432  0.4932  0.3286  0.4794
              2         24        0.2373  0.4872  0.3286  0.4736
              1         96        0.4037  0.6353  0.4569  0.6180
              2         96        0.4003  0.6327  0.4519  0.6153
              1         168       0.4331  0.6581  0.4750  0.6381
              2         168       0.4349  0.6595  0.4764  0.6395
MAE           1         24        0.2378  0.4876  0.3079  0.4740
              2         24        0.2318  0.4814  0.3023  0.4680
              1         96        0.5229  0.7231  0.4732  0.7033
              2         96        0.4096  0.6400  0.4407  0.6225
              1         168       0.4360  0.6603  0.4601  0.6402
              2         168       0.4360  0.6603  0.4598  0.6402

In [12]:
patchtst_df_unscaled.round(4)

MSE       RMSE        MAE     RSE
Loss_function Iteration Pred_len                                          
MSE           1         24        18229302.0  4269.5786  2643.6431  0.2123
              2         24        17448062.0  4177.0879  2625.8184  0.2077
              1         96        31905132.0  5648.4629  3713.9365  0.2813
              2         96        34005252.0  5831.4023  3790.4197  0.2904
              1         168       34134564.0  5842.4795  3888.2668  0.2911
              2         168       35157356.0  5929.3638  3919.8013  0.2954
RMSE          1         24        18199538.0  4266.0918  2667.8621  0.2121
              2         24        17552708.0  4189.5952  2641.2788  0.2083
              1         96        32573556.0  5707.3247  3834.9167  0.2842
              2         96        32110734.0  5666.6333  3774.5425  0.2822
              1         168       35332576.0  5944.1211  3983.2620  0.2962
              2         168       35436588.0  5952.8638  3990.8035  0.2966
MAE           1         24        17101788.0  4135.4307  2451.9343  0.2056
              2         24        16840840.0  4103.7593  2404.9397  0.2040
              1         96        37380588.0  6113.9668  3756.4368  0.3045
              2         96        32031288.0  5659.6191  3615.1736  0.2819
              1         168       34436244.0  5868.2402  3779.6672  0.2924
              2         168       34408188.0  5865.8491  3774.6919  0.2923

In [13]:
# Uncomment the following lines if you want to read saved results
#path_dir = './dataset_results'
#csv_name_scaled = 'patchtst_loss_functions_results_scaled.csv'
#csv_name_unscaled = 'patchtst_loss_functions_results_unscaled.csv'

# Average the iterations
ptst_scaled = pd.read_csv(os.path.join(path_dir, csv_name_scaled))
ptst_unscaled = pd.read_csv(os.path.join(path_dir, csv_name_unscaled))

ptst_res_scaled = ptst_scaled.groupby(['Pred_len', 'Loss_function']).mean().drop('Iteration', axis=1)
ptst_res_scaled.round(4)

MSE    RMSE     MAE     RSE
Pred_len Loss_function                                
24       MAE            0.2348  0.4845  0.3051  0.4710
         MSE            0.2389  0.4888  0.3259  0.4751
         RMSE           0.2403  0.4902  0.3286  0.4765
96       MAE            0.4662  0.6815  0.4570  0.6629
         MSE            0.4144  0.6437  0.4564  0.6261
         RMSE           0.4020  0.6340  0.4544  0.6167
168      MAE            0.4360  0.6603  0.4600  0.6402
         MSE            0.4355  0.6599  0.4753  0.6399
         RMSE           0.4340  0.6588  0.4757  0.6388

In [14]:
ptst_res_unscaled = ptst_unscaled.groupby(['Pred_len', 'Loss_function']).mean().drop('Iteration', axis=1)
ptst_res_unscaled.round(4)

MSE       RMSE        MAE     RSE
Pred_len Loss_function                                          
24       MAE            16971314.0  4119.5950  2428.4370  0.2048
         MSE            17838682.0  4223.3333  2634.7307  0.2100
         RMSE           17876123.0  4227.8435  2654.5704  0.2102
96       MAE            34705938.0  5886.7930  3685.8052  0.2932
         MSE            32955192.0  5739.9326  3752.1781  0.2859
         RMSE           32342145.0  5686.9790  3804.7296  0.2832
168      MAE            34422216.0  5867.0447  3777.1796  0.2923
         MSE            34645960.0  5885.9216  3904.0341  0.2933
         RMSE           35384582.0  5948.4924  3987.0327  0.2964

In [15]:
shutil.rmtree("results_loss_scaled") # we do not need this directory and results anymore. If you need - comment this line

# Rename folder
os.rename("results_loss_unscaled", 'robust_unscaled')

# 3. Robust Scaler Informer IT

In [16]:
# Paths to files and data
data_path = os.getcwd() + "/datasets/"

script_path = "./PatchTST-main/PatchTST_supervised/run_longExp.py"

# Arguments that will be used also for file names
model = "Informer"
dataset = 'IT_data.csv'
losses = ["MSE", "RMSE", "MAE"]
country = dataset[:2]

log_dir = f"logs/loss_choice/robust"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

log_file_path = f"{log_dir}/{model}_{country}.log"

In [17]:
# Dynamic variables
pred_lens = ["24", "96", "168"]
seq_len = "96"
lr = "0.0001"
itr = 2  
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Lists to store the results
informer_results_scaled, informer_results_unscaled = [], []

# Log file
with open(log_file_path, "w") as log_file:
    for loss in losses:
        statement_1 = f"\n=== Starting experiments for loss function: {loss} ===\n"
        log_file.write(statement_1)
        print(statement_1)  # Print to .ipynb output cell

        for pred_len in pred_lens:
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2) 
            model_id = f"{country}_{seq_len}_{pred_len}_loss_choice_for_{country}"

            # Arguments for the command
            command = f"""
            python {script_path} \
              --random_seed 2021 \
              --is_training 1 \
              --root_path "{data_path}" \
              --data_path "{dataset}" \
              --model_id {model_id} \
              --model "{model}" \
              --data "custom" \
              --features M \
              --seq_len {seq_len} \
              --label_len 5 \
              --pred_len {pred_len} \
              --e_layers 2 \
              --d_layers 1 \
              --factor 5 \
              --enc_in 3 \
              --dec_in 3 \
              --c_out 3 \
              --des 'Exp' \
              --train_epochs 10 \
              --patience 3 \
              --overlapping_windows \
              --inverse \
              --loss_fnc "{loss}" \
              --scaler_type robust \
              --itr {itr} --batch_size 32 --learning_rate "{lr}"
            """

            # Run the command and capture the output
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture the output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')  # Print in the .ipynb cell
                log_file.write(line)  # Write to the log file

            # Wait for the process to complete
            process.wait()
            
            # Delete the checkpoints folder and all its contents
            shutil.rmtree('./checkpoints' )

            # Extract metrics for each iteration
            iteration_metrics = extract_metrics_from_output(output, itr)
            iteration_metrics_unscaled = extract_metrics_from_output(output, itr, if_scaled=False)

            # Log the extracted metrics and save them
            for iteration, (scaled_metrics, unscaled_metrics) in enumerate(zip(iteration_metrics, iteration_metrics_unscaled), start=1):
                log_file.write(f"\nExtracted Metrics for {country}, pred_len={pred_len}, iteration={iteration}:\n")
                log_file.write(f"Scaled Metrics - MSE: {scaled_metrics[0]}, RMSE: {scaled_metrics[1]}, MAE: {scaled_metrics[2]}, RSE: {scaled_metrics[3]}\n")
                log_file.write(f"Unscaled Metrics - MSE: {unscaled_metrics[0]}, RMSE: {unscaled_metrics[1]}, MAE: {unscaled_metrics[2]}, RSE: {unscaled_metrics[3]}\n")

                # Append the results to the informer_results lists
                metrics_data = [(informer_results_scaled, scaled_metrics), (informer_results_unscaled, unscaled_metrics)]

                for result_list, metrics in metrics_data:
                    result_list.append({
                        'Loss_function': loss,
                        'Pred_len': pred_len,
                        'Iteration': iteration,
                        'MSE': metrics[0],
                        'RMSE': metrics[1],
                        'MAE': metrics[2],
                        'RSE': metrics[3]
                    })


=== Starting experiments for loss function: MSE ===


=== Starting experiments for pred_len: 24 ===

Args in experiment:
Namespace(random_seed=2021, is_training=1, model_id='IT_96_24_loss_choice_for_IT', model='Informer', data='custom', root_path='/vol/fob-vol3/nebenf24/riabchuv/my_work/datasets/', data_path='IT_data.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', overlapping_windows=True, scaler_type='robust', seq_len=96, label_len=5, pred_len=24, inverse=True, loss_fnc='MSE', if_relu=False, fc_dropout=0.05, head_dropout=0.0, patch_len=16, stride=8, padding_patch='end', revin=1, affine=0, subtract_last=0, decomposition=0, kernel_size=25, individual=0, embed_type=0, enc_in=3, dec_in=3, c_out=3, d_model=512, n_heads=8, e_layers=2, d_layers=1, d_ff=2048, moving_avg=25, factor=5, distil=True, dropout=0.05, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers=10, itr=2, train_epochs=10, batch_size=32, patience=3, learning_rate=0.

In [18]:
path_dir = './dataset_results'
csv_name_scaled = 'informer_loss_functions_results_scaled_IT_robust.csv'
csv_name_unscaled = 'informer_loss_functions_results_unscaled_IT_robust.csv'

# Convert the results into a DataFrame and save as CSV
informer_df_scaled = convert_results_into_df(informer_results_scaled, path_dir, csv_name_scaled)
informer_df_unscaled = convert_results_into_df(informer_results_unscaled, path_dir, csv_name_unscaled)
informer_df_scaled.round(4)

MSE    RMSE     MAE     RSE
Loss_function Iteration Pred_len                                
MSE           1         24        0.1000  0.3163  0.1934  0.3993
              2         24        0.0982  0.3134  0.1980  0.3957
              1         96        0.1649  0.4061  0.2679  0.5121
              2         96        0.1700  0.4123  0.2785  0.5200
              1         168       0.1910  0.4370  0.2906  0.5506
              2         168       0.1857  0.4310  0.2878  0.5429
RMSE          1         24        0.1012  0.3181  0.1956  0.4017
              2         24        0.1000  0.3162  0.1991  0.3993
              1         96        0.1658  0.4072  0.2652  0.5136
              2         96        0.1687  0.4107  0.2637  0.5180
              1         168       0.1943  0.4408  0.2918  0.5553
              2         168       0.1740  0.4172  0.2821  0.5255
MAE           1         24        0.0991  0.3149  0.1830  0.3976
              2         24        0.1078  0.3284  0.1878  0.4147
              1         96        0.1948  0.4414  0.2628  0.5567
              2         96        0.1784  0.4223  0.2571  0.5326
              1         168       0.1986  0.4457  0.2743  0.5614
              2         168       0.2064  0.4544  0.2787  0.5724

In [19]:
informer_df_unscaled.round(4)

MSE       RMSE        MAE     RSE
Loss_function Iteration Pred_len                                           
MSE           1         24        1696561.000  1302.5210   857.1823  0.0915
              2         24        1713331.125  1308.9427   885.3687  0.0920
              1         96        3421302.000  1849.6761  1260.5140  0.1302
              2         96        3697298.250  1922.8359  1325.6306  0.1353
              1         168       4675170.500  2162.2144  1425.9861  0.1523
              2         168       4445287.000  2108.3850  1414.0991  0.1485
RMSE          1         24        1772951.375  1331.5222   882.8345  0.0936
              2         24        1705655.875  1306.0076   881.3223  0.0918
              1         96        3241397.750  1800.3882  1225.7509  0.1267
              2         96        3074052.250  1753.2975  1195.1626  0.1234
              1         168       4601502.000  2145.1111  1414.7355  0.1511
              2         168       3916505.500  1979.0162  1351.4399  0.1394
MAE           1         24        1536245.125  1239.4536   781.2439  0.0871
              2         24        1982624.500  1408.0570   824.5411  0.0989
              1         96        3251107.500  1803.0828  1161.2264  0.1269
              2         96        2883676.750  1698.1392  1115.9299  0.1195
              1         168       3670718.000  1915.9117  1241.1049  0.1350
              2         168       4193679.750  2047.8475  1284.9829  0.1443

In [20]:
# Uncomment the following lines if you want to read saved results
#path_dir = './dataset_results'
#csv_name_scaled = 'informer_loss_functions_results_scaled.csv'
#csv_name_unscaled = 'informer_loss_functions_results_unscaled.csv'

# Average the iterations
informer_scaled = pd.read_csv(os.path.join(path_dir, csv_name_scaled))
informer_unscaled = pd.read_csv(os.path.join(path_dir, csv_name_unscaled))

inf_res_scaled = informer_scaled.groupby(['Pred_len', 'Loss_function']).mean().drop('Iteration', axis=1)
inf_res_unscaled = informer_unscaled.groupby(['Pred_len', 'Loss_function']).mean().sort_index().drop('Iteration', axis=1)
inf_res_scaled.round(4)

MSE    RMSE     MAE     RSE
Pred_len Loss_function                                
24       MAE            0.1035  0.3216  0.1854  0.4061
         MSE            0.0991  0.3148  0.1957  0.3975
         RMSE           0.1006  0.3172  0.1974  0.4005
96       MAE            0.1866  0.4319  0.2600  0.5447
         MSE            0.1674  0.4092  0.2732  0.5161
         RMSE           0.1673  0.4090  0.2644  0.5158
168      MAE            0.2025  0.4500  0.2765  0.5669
         MSE            0.1884  0.4340  0.2892  0.5468
         RMSE           0.1842  0.4290  0.2870  0.5404

In [ ]:
inf_res_unscaled.round(4)

MSE       RMSE        MAE     RSE
Pred_len Loss_function                                            
24       MAE            1.604755e+06  1266.7830   791.1026  0.0890
         MSE            1.669194e+06  1291.9119   844.1920  0.0908
         RMSE           1.674064e+06  1293.7100   840.1658  0.0909
96       MAE            3.198224e+06  1786.7822  1147.4320  0.1257
         MSE            3.237661e+06  1799.3419  1226.9742  0.1266
         RMSE           3.360102e+06  1833.0565  1241.0765  0.1290
168      MAE            4.304869e+06  2074.7513  1328.4107  0.1461
         MSE            4.318529e+06  2076.8229  1384.6658  0.1463
         RMSE           4.214063e+06  2051.1109  1365.7545  0.1445

# 4. Robust Scaler PatchTST IT

In [21]:
# Dynamic variables
pred_lens = ["24", "96", "168"]
seq_len = "512"
lr = "0.0001"
model = "PatchTST"
itr = 2  
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# New log file path
log_file_path = f"{log_dir}/{model}_{country}.log"

patchtst_results_scaled, patchtst_results_unscaled = [], []

with open(log_file_path, "w") as log_file:
    for loss in losses:
        statement_1 = f"\n=== Starting experiments for loss function: {loss} ===\n"
        log_file.write(statement_1)
        print(statement_1)  # Print to notebook

        for pred_len in pred_lens:
            statement_2 = f"\n=== Starting experiments for pred_len: {pred_len} ===\n"
            log_file.write(statement_2)
            print(statement_2) 
            model_id = f"{country}_{seq_len}_{pred_len}_loss_choice_for_{country}"

            # Command arguments
            command = f"""
            python {script_path} \
              --random_seed 2021 \
              --is_training 1 \
              --root_path "{data_path}" \
              --data_path "{dataset}" \
              --model_id {model_id} \
              --model "{model}" \
              --data "custom" \
              --features M \
              --seq_len {seq_len} \
              --label_len 5 \
              --pred_len {pred_len} \
              --e_layers 2 \
              --d_layers 1 \
              --factor 5 \
              --enc_in 3 \
              --dec_in 3 \
              --c_out 3 \
              --des 'Exp' \
              --train_epochs 10 \
              --patience 3 \
              --overlapping_windows \
              --inverse \
              --scaler_type robust \
              --loss_fnc "{loss}" \
              --itr {itr} --batch_size 32 --learning_rate "{lr}"
            """

            # Run the command and capture the output
            process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

            # Capture the output in real-time
            output = []
            for line in process.stdout:
                output.append(line)
                print(line, end='')  # Print in the .ipynb cell
                log_file.write(line)  # Write to the log file

            # Wait for the process to complete
            process.wait()

            # Delete the checkpoints folder and all its contents
            shutil.rmtree('./checkpoints' )

            # Extract metrics for each iteration
            iteration_metrics_scaled = extract_metrics_from_output(output, itr)
            iteration_metrics_unscaled = extract_metrics_from_output(output, itr, if_scaled=False)

            # Log the extracted metrics and save them
            for iteration, (scaled_metrics, unscaled_metrics) in enumerate(zip(iteration_metrics_scaled, iteration_metrics_unscaled), start=1):
                log_file.write(f"\nExtracted Metrics for {country}, pred_len={pred_len}, iteration={iteration}:\n")
                log_file.write(f"Scaled Metrics - MSE: {scaled_metrics[0]}, RMSE: {scaled_metrics[1]}, MAE: {scaled_metrics[2]}, RSE: {scaled_metrics[3]}\n")
                log_file.write(f"Unscaled Metrics - MSE: {unscaled_metrics[0]}, RMSE: {unscaled_metrics[1]}, MAE: {unscaled_metrics[2]}, RSE: {unscaled_metrics[3]}\n")

                # Append the results to the informer_results lists
                metrics_data = [(patchtst_results_scaled, scaled_metrics), (patchtst_results_unscaled, unscaled_metrics)]

                for result_list, metrics in metrics_data:
                    result_list.append({
                        'Loss_function': loss,
                        'Pred_len': pred_len,
                        'Iteration': iteration,
                        'MSE': metrics[0],
                        'RMSE': metrics[1],
                        'MAE': metrics[2],
                        'RSE': metrics[3]
                    })


=== Starting experiments for loss function: MSE ===


=== Starting experiments for pred_len: 24 ===

Args in experiment:
Namespace(random_seed=2021, is_training=1, model_id='IT_512_24_loss_choice_for_IT', model='PatchTST', data='custom', root_path='/vol/fob-vol3/nebenf24/riabchuv/my_work/datasets/', data_path='IT_data.csv', features='M', target='OT', freq='h', checkpoints='./checkpoints/', overlapping_windows=True, scaler_type='robust', seq_len=512, label_len=5, pred_len=24, inverse=True, loss_fnc='MSE', if_relu=False, fc_dropout=0.05, head_dropout=0.0, patch_len=16, stride=8, padding_patch='end', revin=1, affine=0, subtract_last=0, decomposition=0, kernel_size=25, individual=0, embed_type=0, enc_in=3, dec_in=3, c_out=3, d_model=512, n_heads=8, e_layers=2, d_layers=1, d_ff=2048, moving_avg=25, factor=5, distil=True, dropout=0.05, embed='timeF', activation='gelu', output_attention=False, do_predict=False, num_workers=10, itr=2, train_epochs=10, batch_size=32, patience=3, learning_rate=

In [22]:
path_dir = './dataset_results'
csv_name_scaled = 'patchtst_loss_functions_results_scaled_IT_robust.csv'
csv_name_unscaled = 'patchtst_loss_functions_results_unscaled_IT_robust.csv'


# Convert the results into a DataFrame and save as CSV
patchtst_df_scaled = convert_results_into_df(patchtst_results_scaled, path_dir, csv_name_scaled)
patchtst_df_unscaled = convert_results_into_df(patchtst_results_unscaled, path_dir, csv_name_unscaled)
patchtst_df_scaled.round(4)

MSE    RMSE     MAE     RSE
Loss_function Iteration Pred_len                                
MSE           1         24        0.1004  0.3168  0.2046  0.4001
              2         24        0.1020  0.3194  0.2026  0.4033
              1         96        0.1649  0.4061  0.2669  0.5121
              2         96        0.1640  0.4049  0.2658  0.5107
              1         168       0.1745  0.4177  0.2787  0.5262
              2         168       0.1746  0.4179  0.2787  0.5265
RMSE          1         24        0.1024  0.3200  0.2085  0.4040
              2         24        0.1014  0.3184  0.2102  0.4021
              1         96        0.1646  0.4057  0.2658  0.5117
              2         96        0.1635  0.4044  0.2646  0.5100
              1         168       0.1740  0.4171  0.2774  0.5255
              2         168       0.1738  0.4169  0.2772  0.5252
MAE           1         24        0.1068  0.3268  0.2015  0.4127
              2         24        0.1501  0.3874  0.1923  0.4892
              1         96        0.1687  0.4107  0.2600  0.5180
              2         96        0.1686  0.4106  0.2602  0.5179
              1         168       0.1782  0.4222  0.2726  0.5318
              2         168       0.1782  0.4222  0.2727  0.5318

In [23]:
patchtst_df_unscaled.round(4)

MSE       RMSE        MAE     RSE
Loss_function Iteration Pred_len                                           
MSE           1         24        1888251.625  1374.1367   946.0014  0.0966
              2         24        1746225.750  1321.4484   910.2934  0.0929
              1         96        3313422.500  1820.2809  1241.3649  0.1281
              2         96        3265158.750  1806.9750  1229.9159  0.1272
              1         168       3830158.000  1957.0790  1328.4673  0.1379
              2         168       3708326.750  1925.7017  1313.2982  0.1356
RMSE          1         24        2111207.750  1452.9995   989.4656  0.1021
              2         24        2039680.500  1428.1738   989.9337  0.1004
              1         96        3247477.000  1802.0758  1229.2659  0.1268
              2         96        3191100.750  1786.3652  1216.0294  0.1257
              1         168       3762426.000  1939.6974  1316.0566  0.1366
              2         168       3631123.500  1905.5507  1299.1057  0.1342
MAE           1         24        1802445.750  1342.5520   882.4846  0.0943
              2         24        1397562.750  1182.1855   716.2701  0.0831
              1         96        3039094.500  1743.2999  1164.7308  0.1227
              2         96        3028741.000  1740.3279  1162.4697  0.1225
              1         168       3552784.750  1884.8832  1258.2855  0.1328
              2         168       3531337.250  1879.1853  1255.2706  0.1324

In [24]:
# Uncomment the following lines if you want to read saved results
#path_dir = './dataset_results'
#csv_name_scaled = 'patchtst_loss_functions_results_scaled.csv'
#csv_name_unscaled = 'patchtst_loss_functions_results_unscaled.csv'

# Average the iterations
ptst_scaled = pd.read_csv(os.path.join(path_dir, csv_name_scaled))
ptst_unscaled = pd.read_csv(os.path.join(path_dir, csv_name_unscaled))

ptst_res_scaled = ptst_scaled.groupby(['Pred_len', 'Loss_function']).mean().drop('Iteration', axis=1)
ptst_res_scaled.round(4)

MSE    RMSE     MAE     RSE
Pred_len Loss_function                                
24       MAE            0.1285  0.3571  0.1969  0.4509
         MSE            0.1012  0.3181  0.2036  0.4017
         RMSE           0.1019  0.3192  0.2093  0.4030
96       MAE            0.1686  0.4107  0.2601  0.5179
         MSE            0.1644  0.4055  0.2664  0.5114
         RMSE           0.1641  0.4050  0.2652  0.5108
168      MAE            0.1782  0.4222  0.2726  0.5318
         MSE            0.1745  0.4178  0.2787  0.5263
         RMSE           0.1739  0.4170  0.2773  0.5254

In [25]:
# Look at MAE loss pred_len=96  -> absolute is smaller and normalized is higher -> because our
# features have different absolute scales, and in this way the feature with higher values dominate the loss
ptst_res_unscaled = ptst_unscaled.groupby(['Pred_len', 'Loss_function']).mean().drop('Iteration', axis=1)
ptst_res_unscaled.round(4)

MSE       RMSE        MAE     RSE
Pred_len Loss_function                                            
24       MAE            1.600004e+06  1262.3688   799.3773  0.0887
         MSE            1.817239e+06  1347.7925   928.1474  0.0947
         RMSE           2.075444e+06  1440.5867   989.6997  0.1012
96       MAE            3.033918e+06  1741.8139  1163.6003  0.1226
         MSE            3.289291e+06  1813.6279  1235.6404  0.1276
         RMSE           3.219289e+06  1794.2205  1222.6476  0.1263
168      MAE            3.542061e+06  1882.0342  1256.7781  0.1326
         MSE            3.769242e+06  1941.3903  1320.8828  0.1368
         RMSE           3.696775e+06  1922.6240  1307.5812  0.1354

In [26]:
shutil.rmtree("results_loss_scaled") # we do not need this directory and results anymore. If you need - comment this line

# Rename folder
os.rename("results_loss_unscaled", 'robust_unscaled_IT')